# Part 3: GraphQA using LangChain
In this notebook, we will use the LangChain library to answer questions on top of the Kùzu graph we
just created in the previous section. The example below uses the OpenAI GPT-3.5 turbo model to generate
Cypher and answer questions via a text-to-Cypher pipeline, but you can use any other model and see
how it performs.

We start by opening a connection to the existing database and loading the `OPENAI_API_KEY` variable
from a local `.env` file.

In [ ]:
# !uv pip install python-dotenv langchain langchain-community langchain-openai

In [ ]:
import os

import kuzu
from dotenv import load_dotenv

# Load OpenAI API key from .env file
load_dotenv()
assert "OPENAI_API_KEY" in os.environ, "Please set OPENAI_API_KEY in the .env file"

db = kuzu.Database("db")
conn = kuzu.Connection(db)

In [ ]:
from langchain.chains import KuzuQAChain
from langchain_community.graphs import KuzuGraph
from langchain_openai import ChatOpenAI

In [ ]:
# Create a graph object for KuzuQAChain and print the schema
graph = KuzuGraph(db)
print(graph.get_schema)

This schema is passed as part of the prompt to the LLM, which is then used to generate the Cypher query.
The following example shows how to the GPT-3.5 turbo model is used for both text-to-Cypher and for
answer generation.

In [ ]:
chain = KuzuQAChain.from_llm(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=os.environ.get("OPENAI_API_KEY")),
    graph=graph,
    verbose=True,
)


We can then ask questions in natural language.

In [ ]:
chain.invoke("How many wines has Roger Voss tasted?")

In [ ]:
chain.invoke("Give me the full name of the customers who purchased wine that was tasted by Roger Voss?")

## Experiment with open source LLMs
You can use different LLMs, including open source ones, for text-to-Cypher and the answer generation stages. See the
[LangChain docs](https://python.langchain.com/v0.2/docs/integrations/graphs/kuzu_db/#use-separate-llms-for-cypher-and-answer-generation)
for such an example.

Open source LLMs can be self-hosted and served on a local endpoint. In
this example, we use a _much_ cheaper locally running `Mistral-7B-OpenOrca-GGUF` model from LMStudio
for text-to-Cypher, followed by OpenAI's GPT-3.5 turbo model for answer generation. We are still able
to call the `ChatOpenAI` class in both cases because LMStudio's local server mimics OpenAI's API endpoints.

Note that cheap and small open source LLMs may not perform as well as the proprietary, general-purpose ones,
so to obtain best performance on Cypher generation (as well as inference from Cypher quuery results),
you may need to fine-tune a more powerful model.

In [ ]:
chain = KuzuQAChain.from_llm(
    qa_llm=ChatOpenAI(base_url="http://localhost:1234/v1", temperature=0, api_key="not_needed"),
    cypher_llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, api_key=os.environ.get("OPENAI_API_KEY")),
    # qa_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=os.environ.get("OPENAI_API_KEY")),
    graph=graph,
    verbose=True,
)

In [ ]:
chain.invoke("Which country has the most wines that score between 80 and 95 points?")

## Next steps
Feel free to experiment with other LLMs and see how they perform on your own data. As the natural
language questions become more complex, it might result in incorrect Cypher generation, no matter
how good the underlying LLM. In such cases, a query rewriting step may be required to provide better
context to the cypher-generating LLM.

This notebook is just the starting point of utilizing knowledge graphs for retrieval and QA tasks. You can
look at more advanced pipelines that utilize agents, memory and routers via the LangChain and LlamaIndex frameworks,
or simply roll out your own abstractions that fit your use case.

Have fun using graphs, and `pip install kuzu`!